In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

# Importing csv files   

patients=pd.read_csv('Data cleaning.patients.csv')
treatments=pd.read_csv('Data cleaning.treatments.csv')
treatments_cut=pd.read_csv('Data cleaning.treatments_cut.csv')
adverse_reactions=pd.read_csv('Data cleaning .adverse_reactions.csv')

# Purpose

The primary objective of this analysis is to determine whether Auralin exhibits comparable effectiveness to Novodra in establishing a baseline HbA1c level.

# Additional information

Insulin resistance varies person to person, which is why both starting median daily dose and ending median daily dose are required, i.e., to
calculate change in dose.

It is important to test drugs and medical products in the people they are meant to help. People of different age, race, sex, and ethnic group
must be included in clinical trials. This diversity is reflected in the patients table.

### Assesing data

In this step, the data is to be understood more deeply. Before implementing methods to clean it

### Manual assesment 

# Import the pandas library
import pandas as pd

# Assuming patients, treatments, treatment_cut, and adverse_reactions are DataFrames

# To manually assess the data, export the files into an Excel spreadsheet for better user interface
with pd.ExcelWriter('clinical_trials.xlsx') as writer:
    patients.to_excel(writer, sheet_name='patients')
    treatments.to_excel(writer, sheet_name='treatments')
    treatments_cut.to_excel(writer, sheet_name='treatment_cut')
    adverse_reactions.to_excel(writer, sheet_name='adverse_reactions')


### finding issues with dataset
 
 Dirty data 
 
Messy data(untidy data)

 . Dirty Data (Data with Quality issues):
Dirty data, also known as low quality data. Low quality data has content issues.


Duplicated data,

Missing Data,

Corrupt Data,

Inaccurate Data,





Table= `patients`
-patient_id= some rows have misspelled name                                          <accuracy issue>
- state= sometime contain full name sometime abbreviation                            <consistency issue>
- Zip_code = column has enties with 4 digit                                          <validity issue>
- address,city,state,zip_code,country,contact = 12 data missing for these columns    <completion issue>
- assigned_sex, zip_code ,birthdate = Incorrect data type                            <validity issue>
- given_name,surname = duplicate enties in the name of john doe                      <accuracy issue>
- weight = one patient has weight of 48 pounds                                                                                    <accuracy issue>
- height = one patient has hight of 27 inches                                        <accuracy issue>


Table = `treatments & treatments_cut`                                                  
-given_name and surname = all the rows are in lower case                              <consistency issue>
-auralin and novodra columns = remove u from auralin and novodra columns              <validity issue>
-auralin and novodra columns = '-'  is treated as null but in python it will be count <validity issue>
-Hba1c_change  = there are missing values                                             <completion issue>
- 1 duplicate entry by the name of joseph dey                                         <accuracy issue>
-hba1c_change = in this column 9 is instead of 4                                      <accuracy issue>

Table = `adverse_reactions`  
given_name and surname = all the rows are in lower case                               <consistency issue>


patients

. Messy Data When the data has structural issue is it known as Messy or untidy data.

Tidy data has the following properties:

Each variable forms a column

Each observation forms a row

Each observational unit forms a table

Table= `patients`
-contact= contains both phone no and email id

Table =`treatments & treatments_cuts`
-auralin =it should be split in to two columns start and end dose
-novodra = it should be split in to two columns start and end dose

Table = `adverse reaction`
-This table should not exist independently

### Programatic assesment

patients.info()

patients['patient_id'].duplicated().sum()

patients[patients.duplicated(subset= ['given_name','surname'])]

patients.describe()

patients[patients['weight']== 48.800000]

patients[patients['height']== 27.000000]

treatments.info()

treatments.duplicated().sum()
treatments[treatments.duplicated()]

treatments.describe()

treatments.sort_values('hba1c_change')

treatments_cut.sort_values('hba1c_change')

adverse_reactions.info()

adverse_reactions.duplicated().sum()

# Note - Assesing is an iterative process

### Always make copy before cleaning

patients_copy=patients.copy()
treatment_copy=treatments.copy()
treatment_cut_copy=treatments_cut.copy() 
adverse_copy=adverse_reactions.copy()

#### first solve completeness issue

patients_copy.fillna('no data available', inplace = True)
# REPLACE MISSINNG DATA WITH NO DATA AVAILABLE


treatment_copy.info()

# SUBTRACT hbalc_start TO hbalc_end TO GET ALL THE MISSING Hba1c_change
treatment_copy['hba1c_changes']=treatment_copy['hba1c_start'] - treatment_copy['hba1c_end']
# test
treatment_copy.info()


treatment_cut_copy['hba1c_changes']=treatment_cut_copy['hba1c_start']-treatment_cut_copy['hba1c_end']
treatment_cut_copy.info()

#### solveing tidyness issue 

#contact column have to fix

# Extract phone numbers and email addresses separately using regular expressions
patients_copy['phone_number'] = patients_copy['contact'].str.extract(r'(\+?\d{1,3}[-.\s]?\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4})')
patients_copy['email'] = patients_copy['contact'].str.extract(r'([a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,})')

# Strip any leading or trailing spaces from the 'email' column
patients_copy['email'] = patients_copy['email'].str.strip()

# Drop the original 'contact' column
patients_copy.drop(columns=['contact'], inplace=True)



import re 

# Convert NaN values to empty strings
patients_copy['email'] = patients_copy['email'].fillna('')

# Extract phone numbers from the beginning of email addresses
patients_copy['phone_number_two'] = patients_copy['email'].apply(lambda x: re.match(r'^\d{3}-\d{3}-\d{4}', str(x)).group() if re.match(r'^\d{3}-\d{3}-\d{4}', str(x)) else np.nan)


# Remove phone numbers from email addresses using regular expressions
patients_copy['email'] = patients_copy['email'].str.replace(r'^\d{3}-\d{3}-\d{4}', '', regex=True)


# Create a new column 'phn-no' and fill it with values from 'phone_number'
patients_copy['phn-no'] = patients_copy['phone_number']

# Replace NaN values in 'phn-no' with values from 'phone_number_two'
patients_copy['phn-no'].fillna(patients_copy['phone_number_two'], inplace=True)

# Drop 'phone_number' and 'phone_number_two' columns
patients_copy.drop(columns=['phone_number', 'phone_number_two'], inplace=True)




patients_copy

#concating treatment_copy and treatment_cut_copy

treatment_copy = pd.concat([treatment_copy,treatment_cut_copy])

treatment_copy.info()
treatment_copy


# aurain and novodra must be in a same column under medicine type 
treatment_copy=treatment_copy.melt(id_vars=['given_name','surname','hba1c_start','hba1c_end','hba1c_change','hba1c_changes'],var_name='medicine type',value_name='doasge_range')

treatment_copy.info()
treatment_copy

#auralin dosage =it should be split in to two columns start and end dose
#novodra dosage = it should be split in to two columns start and end dose
treatment_copy=treatment_copy[treatment_copy['doasge_range'] != '-']


treatment_copy['dosage_start']=treatment_copy['doasge_range'].str.split('-').str.get(0)
treatment_copy['dosage_end']=treatment_copy['doasge_range'].str.split('-').str.get(1)



treatment_copy.drop(columns=['doasge_range','hba1c_change'],inplace=True)

treatment_copy.info()

# in column dosage_start,dosage_end u should not be there  datatype should be int
treatment_copy['dosage_start']=treatment_copy['dosage_start'].str.replace("u","")
treatment_copy['dosage_end']=treatment_copy['dosage_end'].str.replace("u","")
# to change datatype in to int
treatment_copy['dosage_start']=treatment_copy['dosage_start'].astype(int)
treatment_copy['dosage_end']=treatment_copy['dosage_end'].astype(int)

#check if replace is done correctly and data typr change str to int
treatment_copy.info()


#conbining adverse reaction table with treatment table
treatment_copy=treatment_copy.merge(adverse_copy,how='left',on=['given_name','surname'])

treatment_copy

#### solving validity issue

#in patients table column name zip_code has 2 issue first data type is str should be int second some time pin code is of 4 digit but i decide to ignoor this column bcz it does not effect the conclusion that i want to find
#changing birthdate data type object to date time 
patients_copy['birthdate'] = pd.to_datetime(patients_copy['birthdate'])
#check if datatype change or not
patients_copy.info()

#### solving accuracy issue

# changing name from dsvid to david
#patients_copy=patients_copy.replace(patients_copy['given_name']=='Dsvid','David',inplace=True)

# Convert 'given_name' column to lowercase
patients_copy['given_name'] = patients_copy['given_name'].str.lower()
patients_copy['surname'] = patients_copy['surname'].str.lower()

#there are duplicate entry in the name of john doe we have to drop the duplicates
patients_copy.drop_duplicates(subset=['given_name', 'surname'], inplace=True)
patients_copy

#change weight pound to kg
patients_copy['weight']=patients_copy['weight']*0.453592
patients_copy

# convert height inches to meter 
patients_copy['height']=patients_copy['height']/39.37
patients_copy


#due to the units of height and weight change we have to calcualte proper bmi
patients_copy['bmi']=round(patients_copy['weight'] / (patients_copy['height'])**2,2)

from datetime import datetime

# Assuming patients_copy is your DataFrame and 'birthdate' column exists
# Convert 'birthdate' column to datetime format
patients_copy['birthdate'] = pd.to_datetime(patients_copy['birthdate'], errors='coerce')

# Calculate age based on current date
current_date = datetime.now()
patients_copy['age'] = (current_date - patients_copy['birthdate']).dt.days // 365


patients_copy

# there is a duplicate row in column given_name,surname 
treatment_copy[treatment_copy.duplicated(subset= ['given_name','surname'])]

#Drop the duplicate column
treatment_copy = treatment_copy[~treatment_copy.duplicated(subset=['given_name', 'surname'])]
treatment_copy

#### solve consistency issue

#there are many abbreviation of states it was not possible to identify each one of them so i decidi to keep it as it was

treatment_copy.sort_values(by='hba1c_start')

patients_copy

# These are the two files after cleaning that  i am going to use 
patients_copy.to_csv('updated_patients_detailed_table.csv',index=False)

treatment_copy.to_csv('updated_treatments_detailed_table.csv',index=False )